# Executing an external program

* `os.system()` is not secure. Shell executes the program.
* `subprocess` module is more secure, it provides pipe functionality
* `Popen([fullpath,arg1, arg2, arg3, ...])` creates a subprocess executing the program. Program `stdin` and `stdout` is the terminal.
* `Popen` has `stdin, stdout,stderr` paramaters to control program input, output and error.

In [2]:
from subprocess import Popen,PIPE

In [7]:
# output goes to terminal (in Jupyter it is in server, you cannot see it)
p=Popen(["/bin/ls", "-l"])
p.wait()

total 1728
26351621 -rw-r--r-- 1 onur onur  91803 Mar 20 10:52 01-Scope-and-iterators.ipynb
26351622 -rw-r--r-- 1 onur onur 136973 Dec  1  2021 01-Scope-and-iterators.pdf
26351623 -rw-r--r-- 1 onur onur  70448 Mar 21 16:55 02-Environment-and-Libraries.ipynb
26351624 -rw-r--r-- 1 onur onur 101635 Dec  1  2021 02-Environment-and-Libraries.pdf
26351625 -rw-r--r-- 1 onur onur  32263 Mar 21 17:22 03-Regular-Expressions.ipynb
26351626 -rw-r--r-- 1 onur onur  59993 Dec  1  2021 03-Regular-Expressions.pdf
26351627 -rw-r--r-- 1 onur onur  27765 Mar 28 15:51 04-External-Programs-and-Databases.ipynb
26351628 -rw-r--r-- 1 onur onur  86795 Dec  1  2021 04-External-Programs-and-Databases.pdf
26351629 -rw-r--r-- 1 onur onur   7450 Dec  1  2021 05-Design-Patterns.ipynb
26351630 -rw-r--r-- 1 onur onur  57683 Dec  1  2021 05-Design-Patterns.pdf
26351631 -rw-r--r-- 1 onur onur   9830 Dec  1  2021 06-Concurrency.ipynb
26351632 -rw-r--r-- 1 onur onur  67256 Dec  1  2021 06-Concurrency.pdf
26351633 -rw-r--r

0

## Getting input from a file, output to a file

In [8]:
# gets input from the file , outputs to another file
fp = open("/etc/protocols","r")
ofp = open("testfile.txt","w")
p=Popen(["/bin/grep","v6"], stdin=fp, stdout=ofp)
p.wait()
# check testfile.txt on root of Jupyter.

0

In [9]:
# gets input from the file , outputs to another file
fp = open("/etc/protocols","r")
p=Popen(["/bin/grep","v6"], stdin=fp, stdout=PIPE)
print(p.stdout)
for line in p.stdout:
    print(line)
p.wait()

<_io.BufferedReader name=57>
b'hopopt\t0\tHOPOPT\t\t# IPv6 Hop-by-Hop Option [RFC1883]\n'
b'ipv6\t41\tIPv6\t\t# Internet Protocol, version 6\n'
b'ipv6-route 43\tIPv6-Route\t# Routing Header for IPv6\n'
b'ipv6-frag 44\tIPv6-Frag\t# Fragment Header for IPv6\n'
b'ipv6-icmp 58\tIPv6-ICMP\t# ICMP for IPv6\n'
b'ipv6-nonxt 59\tIPv6-NoNxt\t# No Next Header for IPv6\n'
b'ipv6-opts 60\tIPv6-Opts\t# Destination Options for IPv6\n'
b'mobility-header 135 Mobility-Header # Mobility Support for IPv6 [RFC3775]\n'


0

## Pipes

Pipes are virtual communication channels among the programs. They are used as file objects. If data is written on one end, it can be read from the other end. subprocess.PIPE creates a pipe object when it is used in `stdin, stdout, stderr` parameter.

Unix `man ls | grep modification` is an example of a pipe.

Getting output of a program, giving input to it:

In [10]:
p = Popen(["/bin/ls", "-l"], stdout=PIPE)
print(p.stdout.readlines())
p.wait()

[b'total 1732\n', b'-rw-r--r-- 1 onur onur  91803 Mar 20 10:52 01-Scope-and-iterators.ipynb\n', b'-rw-r--r-- 1 onur onur 136973 Dec  1  2021 01-Scope-and-iterators.pdf\n', b'-rw-r--r-- 1 onur onur  70448 Mar 21 16:55 02-Environment-and-Libraries.ipynb\n', b'-rw-r--r-- 1 onur onur 101635 Dec  1  2021 02-Environment-and-Libraries.pdf\n', b'-rw-r--r-- 1 onur onur  32263 Mar 21 17:22 03-Regular-Expressions.ipynb\n', b'-rw-r--r-- 1 onur onur  59993 Dec  1  2021 03-Regular-Expressions.pdf\n', b'-rw-r--r-- 1 onur onur  26019 Mar 28 15:55 04-External-Programs-and-Databases.ipynb\n', b'-rw-r--r-- 1 onur onur  86795 Dec  1  2021 04-External-Programs-and-Databases.pdf\n', b'-rw-r--r-- 1 onur onur   7450 Dec  1  2021 05-Design-Patterns.ipynb\n', b'-rw-r--r-- 1 onur onur  57683 Dec  1  2021 05-Design-Patterns.pdf\n', b'-rw-r--r-- 1 onur onur   9830 Dec  1  2021 06-Concurrency.ipynb\n', b'-rw-r--r-- 1 onur onur  67256 Dec  1  2021 06-Concurrency.pdf\n', b'-rw-r--r-- 1 onur onur  15407 Dec  1  2021 0

0

In [11]:
p = Popen(["/usr/bin/tr","/a-z/","/A-Z/"], stdin=PIPE, stdout=PIPE)
p.stdin.write('hello /usr/bin/tr\n'.encode())
p.stdin.write(b'please capitilize these\n')
p.stdin.close()
for line in p.stdout:
    print(line.decode(),end='')
p.wait()

HELLO /USR/BIN/TR
PLEASE CAPITILIZE THESE


0

## Chain commands

In [13]:
# man ls | grep modification
print("-- man ls | grep modification --")

p = Popen(["/usr/bin/man","ls"], stdout = PIPE)
q = Popen(["/bin/grep","modification"], stdin = p.stdout, stdout = PIPE)
for line in q.stdout:
    print(line.decode(), end='')
p.wait()
q.wait()

print("-- man ls | grep modification | cat -n --")

#man ls | grep modification | cat -n
p = Popen(["/usr/bin/man","ls"], stdout = PIPE)
q = Popen(["/bin/grep","modification"], stdin = p.stdout, stdout = PIPE)
r = Popen(["/bin/cat","-n"], stdin = q.stdout, stdout = PIPE)
for line in r.stdout:
    print(line.decode(), end='')
p.wait()
q.wait()
r.wait()


-- man ls | grep modification --
       -c     with -lt: sort by, and show, ctime (time of last modification of
              change the default of  using  modification  times;  access  time
-- man ls | grep modification | cat -n --
     1	       -c     with -lt: sort by, and show, ctime (time of last modification of
     2	              change the default of  using  modification  times;  access  time


0

Pipe objects is alive until there is still a reader or writer for that pipe. A PIPE created as a `stdin` pamarameter is automatically opened for writing by the current process (one calling Popen). The readers of the pipe (process with `stdin` parameter) gets EOF when the last reader closes the pipe.

Following is a multiple writer example. If current process does not call `p.stdin.close()`, `grep terminal` process will wait until it.

In [18]:
p = Popen(["/bin/grep", "terminal"], stdin = PIPE, stdout = PIPE)
q1 = Popen(["/usr/bin/man","bash"], stdout = p.stdin)
q2 = Popen(["/usr/bin/man","ls"], stdout = p.stdin)
q3 = Popen(["/usr/bin/man","ssh"], stdout = p.stdin)

# close the unused input pipes
p.stdin.close()

for line in p.stdout:
    print(line.decode(), end='')

q1.wait()
q2.wait()
q3.wait()
p.wait()

              'ls' and output is a terminal)
       standard  output is connected to a terminal.  The LS_COLORS environment
       and error are both connected to terminals (as determined by isatty(3)),
              Used by the select compound command to  determine  the  terminal
              coming  from  a terminal.  In an interactive shell, the value is
       -t fd  True if file descriptor fd is open and refers to a terminal.
       terface  supplied  jointly  by  the  operating system kernel's terminal
       the operating system maintains the notion of a current terminal process
       ID is equal to the current terminal process group ID) receive keyboard-
       differs from the terminal's; such processes are immune to keyboard-gen‐
       if the user so specifies with  stty  tostop,  write  to  the  terminal.
       tostop is in effect) the terminal are sent a SIGTTIN  (SIGTTOU)  signal
       by  the  kernel's  terminal  driver, which, unless caught, suspends the
       when

0

## Synchronization and deadlocks

Pipes and subprocesses are difficult to control for a complicated task. Make sure
unused ends of pipes (especially that you write) are closed. You do not `wait` for a subprocess that is blocked on some other thing (I/O or other process). Otherwise your code will wait forever

# Serializing, Storing-Saving Objects

* `pickle` module allows conversion of an arbitrary object into a string representation and vice versa.
* `pickle.dumps(object)` and `pickle.loads(string)` methods are used in conversion

Serialization scenarios include:
* saving/restoring object state on a file or database
* sending an object over network
* calling methods of a remote object (parameters and return value serialized)
* object and application persistent

In [19]:
import pickle

a=[1,2,3,{'a':123,'b':'hello','c':[1]}]
mystr=pickle.dumps(a)
print(mystr)
b=pickle.loads(mystr)
print(b)

b'\x80\x04\x95*\x00\x00\x00\x00\x00\x00\x00]\x94(K\x01K\x02K\x03}\x94(\x8c\x01a\x94K{\x8c\x01b\x94\x8c\x05hello\x94\x8c\x01c\x94]\x94K\x01aue.'
[1, 2, 3, {'a': 123, 'b': 'hello', 'c': [1]}]


User defined classes can also be serialized with `pickle`. All properly defined methods (no lambda) and members of an object are serialized automatically. If methods make external references (call outside methods or access global variables) they are not serialized. If restoring program does not have those definitions, run time error is generated when invoked.

In [20]:
class LList:
    '''Linked list implementation. Iterator reuse is fixed'''
    class Node:
        def __init__(self, v,n):
            self.val, self.next = v, n
        def __str__(self):
            return "( " + str(self.val) + ", " + str(self.next) + " )"
            
    def __init__(self,vals=[]):
        self.head = self.last = None
        for v in vals:
            self.append(v)
    def append(self,v):
        if self.last == None:
            # very first element
            self.head = self.last = LList.Node(v,None)
        else:
            self.last.next = LList.Node(v,None)
            self.last = self.last.next
    def __getitem__(self,no):
        count = 0
        ptr = self.head
        while count < no:
            if ptr:
                ptr = ptr.next   # next
            else:
                raise IndexError
            count += 1
        if ptr:
            return ptr.val
        else:
            raise IndexError
    def __setitem__(self,no,val):
        count = 0
        ptr = self.head
        while count < no:
            if ptr:
                ptr = ptr.next
            else:
                raise IndexError
            count += 1
        if ptr:
            ptr.val=val
            return ptr.val
        else:
            raise IndexError
    def __delitem__(self,no):
        count = 0
        prev = ptr = self.head
        while count < no:
            if ptr:
                prev = ptr
                ptr = ptr.next
            else:
                raise IndexError
            count += 1
        if ptr:
            if ptr is self.head:
                if self.head is self.last:
                    self.head = self.last = None
                else:
                    self.head = self.head.val
            else: 
                if ptr == self.last:
                    self.last = prev
                prev.next = ptr.next
        else:
            raise IndexError
    def __str__(self):
        ret="["
        ptr = self.head
        while True:
            if ptr:
                ret += str(ptr.val)
            else:
                break
            ptr = ptr.next
            if ptr:
                ret += " -> "
        ret += "]"
        return ret
    
    def __iter__(self):
        '''return a brand new iterator'''
        return LListIterator(self)
    
    # yes, nested iterators possible
    class LListIterator:
        def __init__(self,llist):
            self.llist = llist
            self.itptr = llist.head
        def __next__(self):
            if self.itptr == None:
                raise StopIteration
            else:
                val=self.itptr[0]
                self.itptr = self.itptr[1]
                return val
            

            
a = LList([3,5,8,8,7,6,1])

apick = pickle.dumps(a)
print(apick)

# this scenario will not work when apick is send to a different python instance where LList is **not** defined
b = pickle.loads(apick)
print(b)

b'\x80\x04\x95\xb7\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x05LList\x94\x93\x94)\x81\x94}\x94(\x8c\x04head\x94h\x00\x8c\nLList.Node\x94\x93\x94)\x81\x94}\x94(\x8c\x03val\x94K\x03\x8c\x04next\x94h\x07)\x81\x94}\x94(h\nK\x05h\x0bh\x07)\x81\x94}\x94(h\nK\x08h\x0bh\x07)\x81\x94}\x94(h\nK\x08h\x0bh\x07)\x81\x94}\x94(h\nK\x07h\x0bh\x07)\x81\x94}\x94(h\nK\x06h\x0bh\x07)\x81\x94}\x94(h\nK\x01h\x0bNububububububub\x8c\x04last\x94h\x16ub.'
[3 -> 5 -> 8 -> 8 -> 7 -> 6 -> 1]


In [27]:
import struct
# struct L { int x, int y, char a[10]} l;    fwrite(fp, sizeof(struct L), 1, &l);
help(struct.pack)
help(struct.unpack)

print(struct.pack("ii10s", 10, 10, b"hello"))
print(struct.unpack("ii10s",b'\n\x00\x00\x00\n\x00\x00\x00hello\x00\x00\x00\x00\x00'))

Help on built-in function pack in module _struct:

pack(...)
    pack(format, v1, v2, ...) -> bytes
    
    Return a bytes object containing the values v1, v2, ... packed according
    to the format string.  See help(struct) for more on format strings.

Help on built-in function unpack in module _struct:

unpack(format, buffer, /)
    Return a tuple containing values unpacked according to the format string.
    
    The buffer's size in bytes must be calcsize(format).
    
    See help(struct) for more on format strings.

b'\n\x00\x00\x00\n\x00\x00\x00hello\x00\x00\x00\x00\x00'
(10, 10, b'hello\x00\x00\x00\x00\x00')


# Database Access

* `sqlite3` is a module providing simple single file SQL library with same name
* `db = sqlite3.connect(filepath)` returns a database connector
* `cursor = db.cursor()` returns a handle to execute queries
* `cursor.execute(querystring)` will execute the query
* `cursor.fetchone()` , `cursor.fetchall()` returns a single row or list of rows respectively
* A query result can also be iterated.

In [29]:
import sqlite3
try:
    db=sqlite3.connect("mydb.sql3")
    cur = db.cursor()
except Exception as e:
    print("SQL error",e)
try:
    cur.execute("create table student (stid int primary key, name varchar(40), sname varchar(50))")
except:
    print("error ignored") # ignore this error

try:
    cur.execute("insert into student values (12341,'yilmaz','yilar'), (54213,'nalan','nalmayan'), (61231,'hasan','hasmayan'), (63441,'beren','bermeyen')")
    db.commit()    
except Exception as e:
    print("SQL error",e)

error ignored
SQL error UNIQUE constraint failed: student.stid


In [30]:
try:
    cur.execute("select * from student")
    for v in cur:
        print(v)
except Exception as e:
    print("query error",e)
    
    

(12341, 'yilmaz', 'yilar')
(54213, 'nalan', 'nalmayan')
(61231, 'hasan', 'hasmayan')
(63441, 'beren', 'bermeyen')
(55717, 'onur', 'sehitoglu')


In [32]:
stid='XXX\' OR  name like \'%'
stid2='XXX\'; DELETE STUDENT WHERE NAME LIKE \'%'
try:
    cur.execute("select * from student where stid ='" + stid + "'")
    for v in cur:
        print(v)
    print('-------')
    cur.execute("select * from student where stid =?", (stid,))
    for v in cur:
        print(v)
    print('-------')
    cur.execute("select * from student where stid ='" + stid2 + "'")
except Exception as e:
    print("query error",e)

(12341, 'yilmaz', 'yilar')
(54213, 'nalan', 'nalmayan')
(61231, 'hasan', 'hasmayan')
(63441, 'beren', 'bermeyen')
(55717, 'onur', 'sehitoglu')
-------
-------
query error You can only execute one statement at a time.
